In [7]:
from supabase import create_client,Client
import os
# get environment variables from .env file
import os

def load_env():
    with open('.env', 'r') as f:
        for line in f:
            if line.strip():
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

load_env()


# get environment variables
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)



In [58]:
# functions to load data
import json
import requests

def get_translations(count):
    url = 'https://read.84000-translate.org/translations.json?api-version=0.4.0'
    response = requests.get(url)
    works = json.loads(response.text)
    # only return the required number of translations
    return works['work'][:count]

def get_work(Work_id):
    link = f"../../data-json/{Work_id}.json"
    print("loading file",link)
    # fet the json data from the local file
    with open(link, 'r') as file:
        work = json.loads(file.read())
    return work

def make_titlte(work,WorkId):
    titles = work['work'][0]['title']
    titles_dicts = [dict({'work_uuid': WorkId,
                          'content': title['content'],
                          'titleType': title['titleType'],
                          'language': title['language'],
                          'titleMigrationId': title['titleMigrationId']}) for title in titles]
    return titles_dicts

def make_passages(work,WorkId):
    passages = work['translation']['passage']
    passage_dict = [dict({'work_uuid': WorkId,
                          'content': passage['content'],
                          'segmentationType': passage['segmentationType'],
                          'xmlId': passage['xmlId'],
                          'parentId': passage['parentId'],
                          'passageLabel': passage['passageLabel'],
                          'passageSort': passage['passageSort']}) for passage in passages]
    return passage_dict

def make_work(Current_Work,translation):
    title = translation['work'][0]['title'][0]['content']
    work_dict = dict({'xmlId': Current_Work['workId'],
                      'url': Current_Work['url'],
                      'type': Current_Work['workType'],
                      'toh': Current_Work['catalogueWorkIds'],
                      'title': title,
                      'migrationJson': translation})
    return work_dict

In [72]:
# upsert into supabase
def store_work(work_dict):
    # check if the work already exists with the same xmlId
    # if exists return the workId
    # else insert the work and return the workId
    does = supabase.table('works').select('uuid').eq('xmlId',work_dict['xmlId']).execute()
    display(does.data[0])
    if (does.data[0]):
        print("Work Already Exists")
        return does.data[0]['uuid']
    else :
        try:
            work_data,count = supabase.table('works').insert(work_dict).execute()
            return work_data[1][0]['uuid']
        except Exception as e:
            print ("Error Occured",e)
            return None
    
    
def store_titles(titles_dicts):
    try:
        title_data,count = supabase.table('titles').upsert(titles_dicts).execute()
        print("Added",len(title_data[1]))
    except Exception as e:
        print("Error Occured",e)
        # If error occurs delete all just inserted data
        try:
            if (title_data[1]):
                for inserts in title_data[1]:
                    print(inserts)
                    data_temp,count_temp = supabase.table('titles').delete().eq('id',inserts['id']).execute()
                    print('deleted',data_temp[1])
        except IndexError:
            print("No Data to Delete")
            
def store_passages(passage_dict):
    try:
        passage_data,count = supabase.table('segments').upsert(passage_dict).execute()
        print("Added",len(passage_data[1]))
    except Exception as e:
        print("Error Occured",e)
        # If error occurs delete all just inserted data
        try:
            if (passage_data[1]):
                for inserts in passage_data[1]:
                    print(inserts)
                    data_temp,count_temp = supabase.table('passages').delete().eq('id',inserts['id']).execute()
                    print('deleted',data_temp[1])
        except IndexError:
            print("No Data to Delete")


In [73]:
# Excuter 
# 1. get required  number of translations
# 2. Load the data into Supabase and get back the workId
# 3. Get the work data from the local file
# 4. Get the titles and passages
# 5. Create Title and passage dictionary with WorkId

def load_data(translation_count):
    works = get_translations(translation_count)
    for work in works:
        WorkId = work['workId']
        print("Processing current:",WorkId)
        translation = get_work(WorkId)
        work_dict = make_work(work,translation)
        print("Loading Work")
        display(work_dict)
        work_uuid = store_work(work_dict)
        print("Work UUID",work_uuid)
        titles_dicts = make_titlte(translation,work_uuid)
        print("Loading Titles")
        display(titles_dicts)
        passage_dict = make_passages(translation,work_uuid)
        print("Loading Passages")
        display(passage_dict)
        
        # if ( translation && titles_dicts && passage_dict):
    #         store_titles(titles_dicts)
    #         store_passages(passage_dict)
    
    return


In [74]:
load_data(1)

Processing current: UT22084-001-001
loading file ../../data-json/UT22084-001-001.json
Loading Work


{'xmlId': 'UT22084-001-001',
 'url': '/translation/UT22084-001-001.json?api-version=0.4.0&annotate=true',
 'type': 'eft:translation',
 'toh': 'toh1-1',
 'title': 'The Chapter on Going Forth',
 'migrationJson': {'modelType': 'translation',
  'apiVersion': '0.4.0',
  'publicationVersion': '1.37.9',
  'publicationStatus': '1',
  'publicationDate': '2018-10-31',
  'cacheKey': '2024-04-26-14-26-35-2024-05-22-09-01-52-2-23-4',
  'work': [{'workId': 'UT22084-001-001',
    'workType': 'eft:translation',
    'url': '/translation/UT22084-001-001.json?api-version=0.4.0&annotate=false',
    'htmlUrl': 'https://read.84000.co/translation/UT22084-001-001.html',
    'title': [{'titleType': 'eft:mainTitle',
      'language': 'en',
      'content': 'The Chapter on Going Forth',
      'titleMigrationId': 'toh1-1/eft:mainTitle/en/1'},
     {'titleType': 'eft:mainTitle',
      'language': 'Sa-Ltn',
      'content': 'Pravrajyāvastu',
      'titleMigrationId': 'toh1-1/eft:mainTitle/Sa-Ltn/1'},
     {'titleTy

{'uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb'}

Work Already Exists
Work UUID 929b144e-116f-4bd0-b9ae-f14fa1b5cdcb
Loading Titles


[{'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': 'The Chapter on Going Forth',
  'titleType': 'eft:mainTitle',
  'language': 'en',
  'titleMigrationId': 'toh1-1/eft:mainTitle/en/1'},
 {'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': 'Pravrajyāvastu',
  'titleType': 'eft:mainTitle',
  'language': 'Sa-Ltn',
  'titleMigrationId': 'toh1-1/eft:mainTitle/Sa-Ltn/1'},
 {'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': 'རབ་ཏུ་འབྱུང་བའི་གཞི།',
  'titleType': 'eft:mainTitle',
  'language': 'bo',
  'titleMigrationId': 'toh1-1/eft:mainTitle/bo/1'},
 {'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': 'rab tu ’byung ba’i gzhi',
  'titleType': 'eft:mainTitle',
  'language': 'Bo-Ltn',
  'titleMigrationId': 'toh1-1/eft:mainTitle/Bo-Ltn/1'},
 {'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': 'Chapters on Monastic Discipline, The Chapter on Going Forth',
  'titleType': 'eft:mainTitleOutsideCatalogueSection',
  'language

Loading Passages


[{'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': 'Summary',
  'segmentationType': 'Passage',
  'xmlId': 'summary',
  'parentId': 'summary',
  'passageLabel': 's.',
  'passageSort': 796},
 {'work_uuid': '929b144e-116f-4bd0-b9ae-f14fa1b5cdcb',
  'content': '“The Chapter on Going Forth” is the first of seventeen chapters in The Chapters on Monastic Discipline, a four-volume work that outlines the statutes and procedures that govern life in a Buddhist monastic community. This first chapter traces the development of the rite by which postulants were admitted into the monastic order, from the Buddha Śākyamuni’s informal invitation to “Come, monk,” to the more elaborate “Present Day Rite.” Along the way, the posts of preceptor and instructor are introduced, their responsibilities defined, and a dichotomy between elders and immature novices described. While the heart of the chapter is a transcript of the “Present Day Rite,” the text is interwoven with numerous narrative asid